**Загрузка необходимых библиотек**

In [ ]:
import requests
import json
from copy import deepcopy
IAM_TOKEN = 'ЗДЕСЬ НУЖНО ВВЕСТИ СВОЙ IAM-TOKEN, ПОЛУЧЕННЫЙ В YANDEX DATASPHERE'

#Процедура запроса IAM-токена через командную строку описана в документации Яндекса: https://cloud.yandex.com/en-ru/docs/iam/operations/iam-token/create

**Запрос к обученной модели по одному предложению**

In [ ]:
URL = "https://translate.api.cloud.yandex.net/translate/v2/translate"
HEADERS = {'Content-Type': 'application/json', 'Authorization': f'Bearer {IAM_TOKEN}'}
PAYLOAD = {
 "sourceLanguageCode": SRC_LANG,
 "targetLanguageCode": DST_LANG
}
def translate(text, custom_model_id=None):
     data = deepcopy(PAYLOAD)
     data['texts'] = [text]
     if custom_model_id is not None:
         data['model'] = custom_model_id
     translation_request = requests.post(URL, data=json.dumps(data), headers=HEADERS)
     return translation_request.json()

translate('ЗДЕСЬ САМ ТЕКСТ ДЛЯ ПЕРЕВОДА', custom_model_id='ЗДЕСЬ ВВОДИМ ID МОДЕЛИ, ПОЛУЧЕННОЕ ПО КОМАНДЕ REPORT')

**Запрос к обученной модели через отправку Excel-файла с тестовым сегментом**

In [ ]:
import requests
import json
import pandas as pd
from copy import deepcopy

URL = "https://translate.api.cloud.yandex.net/translate/v2/translate"
HEADERS = {'Content-Type': 'application/json', 'Authorization': f'Bearer {IAM_TOKEN}'}
PAYLOAD = {
 "sourceLanguageCode": SRC_LANG,
 "targetLanguageCode": DST_LANG
}

def translate(text, custom_model_id=None):
    data = deepcopy(PAYLOAD)
    data['texts'] = [text]
    if custom_model_id is not None:
        data['model'] = custom_model_id
    translation_request = requests.post(URL, data=json.dumps(data), headers=HEADERS)
    response = translation_request.json()
    # Assuming the translated text is in the 'translations' field of the response
    return response.get('translations', [{}])[0].get('text', '')

def translate_excel(file_path, column_name, custom_model_id=None, output_file_path='translated.xlsx'):
    # Read the Excel file
    df = pd.read_excel(file_path)

    # Check if the specified column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the Excel file.")

    # Translate each entry in the column and add to a new list
    translations = []
    for text in df[column_name]:
        translated_text = translate(text, custom_model_id)
        translations.append(translated_text)

    # Add the translations as a new column
    df['Yandex trained'] = translations

    # Save the DataFrame with the new column to a new Excel file
    df.to_excel(output_file_path, index=False)

    return output_file_path

# Example usage
output_path = translate_excel('НАЗВАНИЕ EXCEL-ФАЙЛА С ТЕСТОВЫМ СЕГМЕНТОМ', 'НАЗВАНИЕ КОЛОНКИ С ТЕСТОВЫМ СЕГМЕНТОМ', custom_model_id='ЗДЕСЬ ВВОДИМ ID МОДЕЛИ, ПОЛУЧЕННОЕ ПО КОМАНДЕ REPORT')
print(f"Translations saved to {output_path}")